# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8500762fa0>
├── 'a' --> tensor([[-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789]])
└── 'x' --> <FastTreeValue 0x7f8500762f70>
    └── 'c' --> tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                        [ 1.4746, -0.8145, -0.0257,  0.6085],
                        [-0.1082, -0.8685, -0.0549,  1.0889]])

In [4]:
t.a

tensor([[-0.3719, -0.0617, -0.7929],
        [-0.8787, -0.8276,  0.0789]])

In [5]:
%timeit t.a

60.3 ns ± 0.0412 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8500762fa0>
├── 'a' --> tensor([[ 2.0602, -1.0968, -1.6932],
│                   [-0.6504, -0.3734, -0.2428]])
└── 'x' --> <FastTreeValue 0x7f8500762f70>
    └── 'c' --> tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                        [ 1.4746, -0.8145, -0.0257,  0.6085],
                        [-0.1082, -0.8685, -0.0549,  1.0889]])

In [7]:
%timeit t.a = new_value

62.1 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3719, -0.0617, -0.7929],
               [-0.8787, -0.8276,  0.0789]]),
    x: Batch(
           c: tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889]]),
       ),
)

In [10]:
b.a

tensor([[-0.3719, -0.0617, -0.7929],
        [-0.8787, -0.8276,  0.0789]])

In [11]:
%timeit b.a

54.8 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0166, -0.7335, -2.7005],
               [-1.0974, -0.6897,  1.2315]]),
    x: Batch(
           c: tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 19.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 385 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8500762670>
├── 'a' --> tensor([[[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]],
│           
│                   [[-0.3719, -0.0617, -0.7929],
│                    [-0.8787, -0.8276,  0.0789]]])
└── 'x' --> <FastTreeValue 0x7f8500762d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 59.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8453429fa0>
├── 'a' --> tensor([[-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789],
│                   [-0.3719, -0.0617, -0.7929],
│                   [-0.8787, -0.8276,  0.0789]])
└── 'x' --> <FastTreeValue 0x7f8453429eb0>
    └── 'c' --> tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                        [ 1.4746, -0.8145, -0.0257,  0.6085],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.2 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.8453, -1.3067, -0.5482, -2.0179],
                       [ 1.4746, -0.8145, -0.0257,  0.6085],
                       [-0.1082, -0.8685, -0.0549,  1.0889]],
              
                      [[ 0.8453, -1.3067, -0.5482, -2.0179],
                       [ 1.4746, -0.8145, -0.0257,  0.6085],
                       [-0.1082, -0.8685, -0.0549,  1.0889]],
              
                      [[ 0.8453, -1.3067, -0.5482, -2.0179],
                       [ 1.4746, -0.8145, -0.0257,  0.6085],
                       [-0.1082, -0.8685, -0.0549,  1.0889]],
              
                      [[ 0.8453, -1.3067, -0.5482, -2.0179],
                       [ 1.4746, -0.8145, -0.0257,  0.6085],
                       [-0.1082, -0.8685, -0.0549,  1.0889]],
              
                      [[ 0.8453, -1.3067, -0.5482, -2.0179],
                       [ 1.4746, -0.8145, -0.0257,  0.6085],
                       [-0.1082, -0.8685, -0.0549,  1.0889]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 53.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889],
                      [ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889],
                      [ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889],
                      [ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889],
                      [ 0.8453, -1.3067, -0.5482, -2.0179],
                      [ 1.4746, -0.8145, -0.0257,  0.6085],
                      [-0.1082, -0.8685, -0.0549,  1.0889],
                      [ 0.8453, -1.3067, -0.5482, -2.0179],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 803 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
